In [2]:
# !pip install trulens_eval
# !pip install llama-index
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-embeddings-instructor

In [3]:
import os
from OPENAI_KEY import OPENAI_KEY
os.environ["OPENAI_API_KEY"] =  OPENAI_KEY

In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files = ["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
from llama_index.core import Document

document = Document(text = "\n\n".join([doc.text for doc in documents]))

In [9]:
from llama_index.llms.openai import OpenAI
from utils import build_sentence_window_index,get_sentence_window_query_engine

llm = OpenAI(model="gpt-3.5-turbo", temperature = 0.1)

sentence_index = build_sentence_window_index(
    document,
    llm, 
    embed_model = "local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

sentence_window_engine = get_sentence_window_query_engine(sentence_index=sentence_index)

In [15]:
output = sentence_window_engine.query("How do you create your AI portfolio?")
output.response

'You create your AI portfolio by building a collection of projects that demonstrate a progression of skills in the field. This can involve working on various AI projects that showcase your abilities and growth in areas such as coding, problem-solving, and data analysis.'

In [16]:
# Dashboard
import nest_asyncio
nest_asyncio.apply()

In [29]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

## Answer relevance

In [34]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer relevance"
).on_input().on_output()

✅ In Answer relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Context relevance (query -> RAG)

In [31]:
from trulens_eval import TruLlama # integration between trulens and llamaindex

context_selection = TruLlama.select_source_nodes().node.text

In [32]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


### Groundedness

In [36]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness( groundedness_provider=provider)
f_groundedness = (
    Feedback(
        grounded.groundedness_measure_with_cot_reasons,
        name="Groundedness",
    )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wongs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


 ## Evaluation using RAG triads

In [40]:
from trulens_eval import TruLlama, FeedbackMode

tru_recorder = TruLlama(
    app= sentence_window_engine, ## LLamaindex app
    app_id= "Basic RAG",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [43]:
eval_questions = []
with open('./eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and hwo does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [44]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Groundedness,Context Relevance,Answer relevance,Groundedness_calls,Context Relevance_calls,Answer relevance_calls,latency,total_tokens,total_cost
0,Basic RAG,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_254f46a7f9ffb3a3cd812d57c8eea74f,"""What are the keys to building a career in AI?""","""The keys to building a career in AI involve l...",-,"{""record_id"": ""record_hash_254f46a7f9ffb3a3cd8...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-13T11:09:16.601773"", ""...",2024-04-13T11:09:23.739470,1.000000,0.90,0.8,[{'args': {'source': 'Chapter 6: Building a Po...,[{'args': {'question': 'What are the keys to b...,[{'args': {'prompt': 'What are the keys to bui...,7,540,0.000825
1,Basic RAG,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_3295188ff5e9a7ee34890eba47be1724,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_3295188ff5e9a7ee348...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-13T11:09:24.468788"", ""...",2024-04-13T11:09:33.428982,0.600000,0.75,0.9,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,[{'args': {'prompt': 'How can teamwork contrib...,8,706,0.001096
2,Basic RAG,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_f131aa5f24433fec4fc9c521404ce495,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",-,"{""record_id"": ""record_hash_f131aa5f24433fec4fc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-13T11:09:33.897115"", ""...",2024-04-13T11:09:42.275762,0.466667,0.75,0.9,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,[{'args': {'prompt': 'What is the importance o...,8,555,0.000869
3,Basic RAG,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_98ecf23d2ab0b980b1ffa7734bf439b7,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_98ecf23d2ab0b980b1f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-13T11:09:42.836928"", ""...",2024-04-13T11:09:50.060251,1.000000,0.60,1.0,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,[{'args': {'prompt': 'What are some good habit...,7,548,0.000840
4,Basic RAG,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_76408c8b4326a411b4de87edcb95709b,"""How can altruism be beneficial in building a ...","""Helping others and lifting them up during one...",-,"{""record_id"": ""record_hash_76408c8b4326a411b4d...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-04-13T11:09:50.560602"", ""...",2024-04-13T11:09:58.080106,0.000000,0.20,0.8,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,[{'args': {'prompt': 'How can altruism be bene...,7,509,0.000774


In [47]:
import pandas as pd

records[['input', 'output']+feedback]

,input,output,Context Relevance,Groundedness,Answer relevance
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI involve l...",0.90,1.000000,0.8
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",0.75,0.600000,0.9
2,"""What is the importance of networking in AI?""","""Networking in AI is crucial as it can provide...",0.75,0.466667,0.9
3,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",0.60,1.000000,1.0
4,"""How can altruism be beneficial in building a ...","""Helping others and lifting them up during one...",0.20,0.000000,0.8
5,"""What is imposter syndrome and hwo does it rel...","""Imposter syndrome is a phenomenon where indiv...",0.90,0.600000,0.9
6,"""Who are some accomplished individuals who hav...","""Former Facebook COO Sheryl Sandberg, U.S. fir...",1.00,1.000000,1.0
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to l...",0.75,1.000000,0.9
8,"""What are some common challenges in AI?""","""Common challenges in AI include the evolving ...",0.50,0.700000,0.9
9,"""Is it normal to find parts of AI challenging?""","""It is normal to find parts of AI challenging.""",0.80,1.000000,1.0


In [48]:
tru.get_leaderboard()

,Context Relevance,Groundedness,Answer relevance,latency,total_cost
app_id,,,,,
Basic RAG,0.715,0.736667,0.91,7.2,0.000916


In [49]:
tru.run_dashboard()

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.89.63.194:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>